In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [87]:
from IPython.display import display, Markdown, Latex

In [ ]:
df = pd.read_csv('data/Clean_Dataset.csv')
df.sample(20)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
print(f"{'column':<20}{'# unique values'}")
print("-"*40)
for column in df.columns:
    print(f"{column:<20}{df[column].nunique()}")

In [ ]:
df.dtypes

In [ ]:
CAT_COLS = [
    'airline',
    'flight',
    'source_city',
    'departure_time',
    'stops',
    'arrival_time',
    'destination_city',
    'class',
]

NUM_COLS = [
    'duration',
    'days_left',
    'price',
]

CAT_COLS_SMALL = CAT_COLS.copy()
CAT_COLS_SMALL.remove("flight")

df_eco = df[df["class"] == "Economy"]
df_bus = df[df["class"] == "Business"]


print(f"{CAT_COLS=}")
print(f"{CAT_COLS_SMALL=}")
print(f"{NUM_COLS=}")

In [ ]:
print("Cat Cols values:")
print()

for column in CAT_COLS:
    print(f"{column.upper()}:")
    print(", ".join(df[column].unique()))
    print()

In [ ]:
# Calculate heights based on unique categories in each categorical column
heights = [df[col].nunique() for col in CAT_COLS_SMALL]
total_height = sum(heights)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

fig, axs = plt.subplots(
    ncols=1,
    nrows=len(CAT_COLS_SMALL),
    figsize=(10, len(CAT_COLS_SMALL)),
    sharex=True,
    gridspec_kw={'height_ratios': heights}  # Use height_ratios to adjust each row's height
)

for column, ax, color in zip(CAT_COLS_SMALL, axs.flatten(), colors):
    order = df[column].value_counts().index    
    sns.countplot(
        df,
        y=column,
        ax=ax,
        order=order,
        color=color,
    )
    
for column, ax, color in zip(CAT_COLS_SMALL, axs.flatten(), colors):
    order = df[column].value_counts().index    
    sns.countplot(
        df_bus,
        y=column,
        ax=ax,
        order=order,
        color="white",
        alpha=0.5
    )

fig.tight_layout()
plt.subplots_adjust(hspace=0.0, wspace=0)

In [ ]:
len(CAT_COLS_SMALL)

In [ ]:
# Calculate heights based on unique categories in each categorical column
heights = [df[col].nunique() for col in CAT_COLS_SMALL]
total_height = sum(heights)
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

def create_bar_plot(df):

    # Create subplots – one column per categorical column
    _, axs = plt.subplots(
        nrows=len(CAT_COLS_SMALL),  # Transpose: Now each row corresponds to a categorical column
        ncols=len(NUM_COLS),        # Transpose: Each column corresponds to a numerical column
        figsize=(15, total_height * 0.3),  # Adjust height dynamically based on total height
        sharex='col',  # Sharing x-axis along each column
        sharey='row',
        gridspec_kw={'height_ratios': heights}  # Use height_ratios to adjust each row's height
    )
    # Iterate over categorical columns using enumerate
    for (row_index, cat_col), color in zip(enumerate(CAT_COLS_SMALL), colors):
        for col_index, num_col in enumerate(NUM_COLS):
            
            # Access the subplot using the current index
            ax = axs[row_index, col_index]
            
            # Create a boxplot for the current combination with numerical categories on y-axis
            sns.boxplot(
                data=df,
                x=num_col,  # Numerical column on x-axis
                y=cat_col,  # Categorical data on y-axis
                ax=ax,
                #hue="class",
                color=color,
            )

    # Adjust layout to remove gaps
    plt.subplots_adjust(hspace=0.0, wspace=0)

    plt.show()


display(Markdown("## Economy Flights"))
create_bar_plot(df_eco)

display(Markdown("## Business Flights"))
create_bar_plot(df_bus)


In [ ]:
fig, axs = plt.subplots(
    ncols=1,
    nrows=len(NUM_COLS),
    figsize=(10, 12),
)

for column, ax in zip(NUM_COLS, axs.flatten()):
    sns.histplot(
        data=df,
        x=column,
        bins=100,
        ax=ax,
        hue="class",
    )
    
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

sns.regplot(df_eco.sample(1000), y="price", x="days_left", ax=ax, marker=".", order=2)
sns.regplot(df_bus.sample(1000), y="price", x="days_left", ax=ax, marker=".", order=1)


In [ ]:
sns.pairplot(data=df.sample(2000), hue="class")

In [ ]:
# Define datasets and plotting parameters
datasets = [
    {'df': df_eco, 'cmap': 'Blues'},
    {'df': df_bus, 'cmap': 'Oranges'},
]

metrics = [
    {'title': 'Count of Flights', 'aggfunc': None, 'values': None},
    {'title': 'Mean Duration', 'aggfunc': 'mean', 'values': 'duration'},
    {'title': 'Mean Price', 'aggfunc': 'mean', 'values': 'price'},
]

# Create subplots
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(15, 10), sharex=True, sharey=True)

# Iterate over datasets and metrics to create heatmaps
for i, dataset in enumerate(datasets):
    df = dataset['df']
    cmap = dataset['cmap']
    
    for j, metric in enumerate(metrics):
        ax = axs[i, j]

        # Compute crosstab
        crosstab_data = pd.crosstab(
            index=df['source_city'],
            columns=df['destination_city'],
            values=df[metric['values']] if metric['values'] else None,
            aggfunc=metric['aggfunc'],
            dropna=False
        ).replace(0, np.nan)
        
        # Plot heatmap
        sns.heatmap(
            crosstab_data,
            cmap=cmap,
            annot=True,
            fmt=".5g",
            cbar=False,
            vmin=crosstab_data.min().min(),
            ax=ax
        )

        # Set titles and labels
        ax.set_title(metric['title'] if i == 0 else None)
        ax.set_xlabel('Destination City' if i == 1 else None)
        ax.set_ylabel('Source City' if j == 0 else None)
        ax.tick_params(left=False, bottom=False, labelrotation=0)

plt.tight_layout()
plt.show()

In [ ]:
for column in CAT_COLS:
    print(df.groupby(column)['price'].mean())
    print('='*25)

In [ ]:
for i in CAT_COLS:
    print(df.groupby(['class', i])['price'].mean())
    print('='*25)

In [ ]:
df.groupby('class')['price'].plot(kind = 'hist', bins = 50)

In [ ]:
sns.heatmap(data = df.loc[:, ['duration', 'days_left', 'price']].corr(), annot = True)

In [ ]:
mask_eco = df.loc[:, 'class'] == 'Economy'
df.loc[mask_eco, :].describe()

In [ ]:
mask_bis = df.loc[:, 'class'] == 'Business'
df.loc[mask_bis, :].describe()